# KNN算法原理

In [1]:
import numpy as np
from math import sqrt
from collections import Counter

def KNN(k, X_train, y_label, x):
    distances = [sqrt(np.sum((x_train - x)**2)) for x_train in X_train] #计算给定待预测点与训练集点的距离
    nearest = np.argsort(distances) #排序得到最近的点
    
    y_topK = [y_label[i] for i in nearest[:k]]
    votes = Counter(y_topK) #统计最近的 k 个点的标签
    
    return votes.most_common(1)[0][0] #返回 x 的标签

# 使用sklearn的KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=k) #输入 k 值

knn_classifier.fit(X_train, y_label) #训练模型

knn_classifier.predict(x) #预测

## 手动评估数据集

通过拆分训练数据集和测试数据集

In [3]:
import matplotlib.pyplot as plt
from sklearn import datasets

iris = datasets.load_iris()

X = iris.data
y = iris.target

In [4]:
#将 X 乱序

shuffle_indexes = np.random.permutation(len(X))

#设定测试数据集占 20%

test_ratio = 0.2
test_size = int(len(X) * test_ratio)

#分割数据集

test_indexes = shuffle_indexes[:test_size]
train_indexes = shuffle_indexes[test_size:]

X_train = X[train_indexes]
y_train = y[train_indexes]

X_test = X[test_indexes]
y_test = y[test_indexes]

In [5]:
#进行训练与测试

from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)

y_predict = knn_classifier.predict(X_test)

In [6]:
#计算下准确率

sum(y_predict == y_test)/len(y_test)

1.0

# sklearn 中的 train_test_split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#可以传入随即种子 random_state，以便调参

# 分类准确度

In [12]:
digits = datasets.load_digits() #手写数字集
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)

y_predict = knn_classifier.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.9944444444444445

# 超参数

超参数：运行机器学习算法之前需要设定好的参数，如KNN中的k，超参数是调参的对象

## 寻找最佳的 k

In [15]:
best_score = 0.0
best_k = -1

for k in range(1,11):
    knn_classifier = KNeighborsClassifier(n_neighbors=k)
    knn_classifier.fit(X_train, y_train)
    y_predict = knn_classifier.predict(X_test)
    score = accuracy_score(y_test, y_predict)
    if score > best_score:
        best_k = k
        best_score = score
        

print("best_k = ", best_k)
print("best_score = ", best_score)

best_k =  4
best_score =  0.9916666666666667


## 是否考虑距离

在训练数据时加入超参数，weights = "distance" 或 "uniform"

对该参数进行上述的调参过程

In [17]:
best_method = ""
best_score = 0.0
best_k = -1
for method in ["uniform", "distance"]:
    for k in range(1,11):
        knn_classifier = KNeighborsClassifier(n_neighbors=k, weights=method)
        knn_classifier.fit(X_train, y_train)
        y_predict = knn_classifier.predict(X_test)
        score = accuracy_score(y_test, y_predict)
        if score > best_score:
            best_k = k
            best_score = score
            best_method = method

print("best_method = ", method)
print("best_k = ", best_k)
print("best_score = ", best_score)

best_method =  distance
best_k =  4
best_score =  0.9916666666666667


### 明可夫斯基距离

$$\left({\sum_{i=1}^n |X_i^\left(a\right)-X_i^\left(b\right)|^p}\right)^{\frac 1p}$$

于是我们得到了超参数 p，我们可以针对明可夫斯距离进行搜索

In [19]:
%%time

best_p = -1
best_score = 0.0
best_k = -1

for k in range(1,11):
    for p in range(1,6):
        knn_classifier = KNeighborsClassifier(n_neighbors=k, weights="distance", p=p)
        knn_classifier.fit(X_train, y_train)
        y_predict = knn_classifier.predict(X_test)
        score = accuracy_score(y_test, y_predict)
        if score > best_score:
            best_k = k
            best_score = score
            best_p = p

print("best_p = ", p)
print("best_k = ", best_k)
print("best_score = ", best_score)

best_p =  5
best_k =  3
best_score =  0.9888888888888889
CPU times: user 11.2 s, sys: 12.1 ms, total: 11.2 s
Wall time: 11.2 s


# 网格搜索

In [20]:
#创建网格
param_grid = [
    {
        'weights':['uniform'],
        'n_neighbors':[i for i in range(1,11)]
    },
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1,11)],
        'p':[i for i in range(1,6)]
    }
]

In [21]:
knn_classifier = KNeighborsClassifier()

In [22]:
%%time

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(knn_classifier, param_grid)

grid_search.fit(X_train, y_train)

CPU times: user 36.8 s, sys: 29.7 ms, total: 36.9 s
Wall time: 36.9 s


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [23]:
grid_search.best_estimator_ #最佳模型

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [24]:
grid_search.best_score_ #最高准确率

0.9860820751064653

In [25]:
grid_search.best_params_ #最优超参数

{'n_neighbors': 1, 'weights': 'uniform'}

In [27]:
knn_classifier = grid_search.best_estimator_ #使用优化模型
y_predict = knn_classifier.predict(X_test)
accuracy_score(y_test, y_predict) #测试集准确率

0.9833333333333333

GridSearchCV 的其他参数：n_jobs为计算机运算的核的数量；verbose为计算过程中的信息，可取verbose = 2

In [30]:
%%time

grid_search = GridSearchCV(knn_classifier, param_grid, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 228 tasks      | elapsed:    3.2s


CPU times: user 424 ms, sys: 40.3 ms, total: 464 ms
Wall time: 4.28 s


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    4.3s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=1, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

# 数据归一化

防止结果被数量差距较大的一方特征所主导

解决方法：将每一个特征映射到同一尺度。
***
最值归一化：把所有数据映射到[0,1] $$x_{scale}=\frac{x-x_{min}}{x_{max}-x_{min}}$$
适用于分布有明显边界的情况，受outlier影响较大
***
均值方差归一化：把所有数据都映射到均值为0方差为1的分布中 $$x_{scale}=\frac{x-x_{mean}}s$$
适用于分布没有明显界限，有可能存在极端值
***
**测试数据集的均值和方差应该使用训练数据集的**

In [31]:
#使用鸢尾花数据集
X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [32]:
from sklearn.preprocessing import StandardScaler

stdscaler = StandardScaler()
stdscaler.fit(X_train) #使用均值方差进行预处理归一化，得到相应的 mean_ 和 scale_

StandardScaler(copy=True, with_mean=True, with_std=True)

In [33]:
#处理训练集和测试集

X_train = stdscaler.transform(X_train)
X_test = stdscaler.transform(X_test)

### K近邻算法的缺点：

1. 效率低下
2. 数据依赖性高
3. 预测结果不具有可解释性
4. 维数灾难